# Day1課題①

クラウドファンディングが成功するか(state)を事前に予測するモデルを構築すること
<br/>使用する説明変数にご注意下さい。
<br/>データ：ks_projects_201801.csvを使用する


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

# データの読み込み

### 2. Scikit-learnを用いてロジスティック回帰を実装してみましょう
- 学習後の重みw0, w1, ..., w6を表示してみましょう
- 実際に予測してみて，対数尤度・正答率を表示してみましょう
- 最後に正解と予測値のクロス集計の結果を表示してみましょう

In [2]:
df_house = pd.read_csv("./ks-projects-201801.csv")[['state','pledged','goal','backers','usd pledged','usd_pledged_real','usd_goal_real']]
df_house['state'] = df_house['state'] == "successful" 

# データ
display(df_house.head())
display(df_house.describe())

,state,pledged,goal,backers,usd pledged,usd_pledged_real,usd_goal_real
0,False,0.0,1000.0,0,0.0,0.0,1533.95
1,False,2421.0,30000.0,15,100.0,2421.0,30000.00
2,False,220.0,45000.0,3,220.0,220.0,45000.00
3,False,1.0,5000.0,1,1.0,1.0,5000.00
4,False,1283.0,19500.0,14,1283.0,1283.0,19500.00


,pledged,goal,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,9.682979e+03,4.908079e+04,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,9.563601e+04,1.183391e+06,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,0.000000e+00,1.000000e-02,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,3.000000e+01,2.000000e+03,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,6.200000e+02,5.200000e+03,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,4.076000e+03,1.600000e+04,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.033899e+07,1.000000e+08,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [3]:
#欠損値がないか確認
df_house.isnull().sum()

state                  0
pledged                0
goal                   0
backers                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

In [4]:
#欠損値：Fare (testデータ)
#testのFareを埋める
df_house['usd pledged'] = df_house['usd pledged'].fillna("0")

In [5]:
#欠損値がないことを確認
df_house.isnull().sum()

state               0
pledged             0
goal                0
backers             0
usd pledged         0
usd_pledged_real    0
usd_goal_real       0
dtype: int64

In [6]:
y = df_house["state"].values
X = df_house.drop('state', axis=1).values

#確率的勾配降下法
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

# 重みを取得して表示
w0 = clf.intercept_[0]
w1 = clf.coef_[0, 0]
w2 = clf.coef_[0, 1]
w3 = clf.coef_[0, 2]
w4 = clf.coef_[0, 3]
w5 = clf.coef_[0, 4]
w6 = clf.coef_[0, 5]
print('w0 = {:.3f}, w1 = {:.3f}, w2 = {:.3f}, w3 = {:.3f}, w4 = {:.3f}, w5 = {:.3f}, w6 = {:.3f}'.format(w0, w1, w2, w3, w4, w5, w6))

w0 = 605.359, w1 = 2171.616, w2 = -1676.114, w3 = -911.120, w4 = -168.918, w5 = -1032.052, w6 = 967.454


In [7]:
# ラベルを予測
y_est = clf.predict(X)

# 対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est)))

# 正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))

対数尤度 = -0.534
正答率 = 98.455%


In [9]:
# 予測値と正解のクロス集計
conf_mat = pd.DataFrame(confusion_matrix(y, y_est), 
                        index=['正解 = クラウドファンディング失敗', '正解 = クラウドファンディング成功'], 
                        columns=['予測 = クラウドファンディング失敗', '予測 = クラウドファンディング成功'])
conf_mat

,予測 = クラウドファンディング失敗,予測 = クラウドファンディング成功
正解 = クラウドファンディング失敗,238932,5773
正解 = クラウドファンディング成功,77,133879
